In [1]:
import numpy as np
import pandas as pd
import os, itertools, csv
from bs4 import BeautifulSoup
import re
from pytrends.request import TrendReq
from datetime import *
import time
import nltk
from termcolor import colored

# load the dataset
dataset = pd.read_csv('./data/train.csv')
dataset.head()

start = 18000
end = 21000

In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('wordnet')
stop_words = set(stopwords.words("english"))

def get_category(text: str) -> list:
    res = list()
    soup = BeautifulSoup(text, "html.parser")
    cats = soup.find_all('a', {'href': re.compile('/category/*')})
    for cat in cats:
        res.append(cat.get_text().lower())
    lem = WordNetLemmatizer()
    res = [lem.lemmatize(word) for word in res if not word in  
        stop_words]
    return list(set(res))

res = []
for i in range(0, dataset.shape[0]):
    res.append(get_category(dataset['Page content'][i]))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yuwei\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
cats = res
cats

[['asteroid', 'world', 'earth', 'u.s.', 'space', 'challenge'],
 ['patent-lawsuit theater',
  'apps and software',
  'patent lawsuit theater',
  'open source',
  'opn pledge',
  'software patents',
  'u.s.',
  'google',
  'tech'],
 ['nfl draft', 'espn', 'sport', 'entertainment', 'television', 'nfl'],
 ['youtube', 'watercooler', 'sport', 'video'],
 ['instagram', 'sport', 'entertainment', 'instagram video', 'nfl'],
 ['startup',
  'lobbying',
  'mobile',
  'world',
  'internet service provider',
  'lobbyist',
  'government',
  'u.s.',
  'internet',
  'politics'],
 ['music', 'entertainment', 'parody', 'funny', 'iggy azalea', 'video'],
 ['television', 'bill cosby', 'entertainment'],
 ['vending machines',
  'mobile',
  'apps and software',
  'vending machine',
  'gadget',
  'tech'],
 ['business', 'the new york times', 'medium', 'paywall'],
 ['moon', 'world', 'saturn', 'astronomy', 'space'],
 ['humor', 'mad men', 'watercooler', 'photography', 'television'],
 ['apps and software', 'nevada', 'u.

In [4]:
from pytrends.request import TrendReq
from datetime import *
import time
import requests
import random

In [5]:
from termcolor import colored

def get_proxy_list():
    print(colored("Update proxies...", "blue"))
    proxies = requests.get("http://list.didsoft.com/get?email=yuwei.shih.tw@gmail.com&pass=73frss&pid=httppremium&showcountry=no&country=SG")
    proxies = proxies.text
    #print(proxies.split("\n")[13:])
    proxies = proxies.split("\n")[:]
    #print(proxies)
    proxy_list = []
    for proxy in proxies:
        proxy_list.append("http://" + proxy)
    proxy_list.remove("http://")
    random.shuffle(proxy_list)
    return proxy_list

# Test
#print(get_proxy_list())
proxy_list = get_proxy_list()
print(int(str(datetime.now()).split(":")[1]) % 30 == 0)
print(proxy_list)

Update proxies...
False
['http://157.245.144.77:8888', 'http://180.210.201.54:3128', 'http://159.89.201.15:8080', 'http://139.59.109.156:3128', 'http://206.189.45.191:80', 'http://157.245.56.142:8080', 'http://180.210.201.55:3129', 'http://178.128.100.74:8080', 'http://157.245.153.49:3128', 'http://157.245.57.147:8080', 'http://178.128.62.211:8080', 'http://139.59.99.234:3128', 'http://54.255.188.140:80', 'http://167.71.192.11:8080', 'http://128.199.90.230:8118', 'http://157.245.192.16:8118', 'http://167.71.200.182:8118', 'http://150.95.25.110:3128', 'http://45.77.174.156:3128', 'http://159.65.142.133:8888', 'http://157.245.48.174:8080', 'http://119.81.71.27:8123', 'http://163.44.152.184:3128', 'http://157.245.60.143:8080', 'http://149.129.57.233:8118', 'http://178.128.106.137:8080', 'http://209.97.173.209:8080', 'http://149.28.139.109:3128', 'http://157.245.48.206:8080', 'http://178.128.50.159:8080', 'http://139.59.101.223:8080', 'http://178.128.27.168:8080', 'http://165.22.63.194:808

In [40]:
def get_date(text: str) -> datetime:
    soup = BeautifulSoup(text)
    selector = "time"
    date = [i.text for i in soup.select(selector)][0]
    date = date.split()
    date = str(date[0] + ' ' + date[1])

    d = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return d

def get_trend(batch: list, t_start: str, t_end: str) -> float:
    global proxy_list
    if int(str(datetime.now()).split(":")[1]) % 30 == 0:
        proxy_list = get_proxy_list()
    g_trends = TrendReq(tz=360, timeout=(5,25), proxies=proxy_list, retries=2, backoff_factor=0.1)
    g_trends.build_payload(batch, timeframe = t_start+' '+t_end, geo='', gprop='')
    trend = g_trends.interest_over_time()
    trend = trend.drop(columns=['isPartial'])
    print(trend)
    max = 0
    if trend.shape[0] > 0:
        for col in range(trend.shape[1]):
            print(trend.iloc[:15, col].values)
            print(trend.iloc[15:, col].values)
            past = np.sum(trend.iloc[:15, col].values)
            now = np.sum(trend.iloc[15:, col].values)
            if past > 0:
                print(now/past)
                if now/past > max:
                    max = now/past
            else:
                if 2 > max:
                    max = 2
    return max

In [42]:
def trend_num(batch: list, pub_time: datetime):
    y_start = str(pub_time+ timedelta(days=-15)).split(' ')[0]
    y_end = str(pub_time + timedelta(days=14)).split(' ')[0]
    print(y_start, y_end)
    global proxy_list
    for i in range(0, 100):
        try:
            return get_trend(batch, y_start, y_end)
            break
        except Exception as e:
            print(e)
            print("Retry step: " + str(i))
            proxy_list = get_proxy_list()
    
def trend_rate(batches: list, pub_time: datetime):
    tmp_rate = [0]
    for batch in batches:
        tmp_rate.append(float(trend_num(batch, pub_time)))
    return max(tmp_rate)
    
def export(check_point):
    g_trend = pd.DataFrame(res)
    g_trend.columns = ["trend_rate"]
    ID = [i for i in range(start, check_point)]
    g_trend['Id'] = ID
    g_trend = g_trend.set_index('Id')
    g_trend.to_csv("trend_%s_%s.csv" % (str(start), str(check_point)))
    
def make_batch(cat_list):
    batch_num = int(len(cat_list)/4)
    if len(cat_list)%4 > 0:
        batch_num = batch_num + 1

    return np.array(np.array_split(cat_list, batch_num)).tolist()

In [43]:
date = get_date(dataset["Page content"][start+1])
batches = make_batch(cats[18001])
print(trend_rate(batches, date))

2014-03-18 2014-04-16
            globe  matchstick  watercooler
date                                      
2014-03-18     93           1            0
2014-03-19     95           1            0
2014-03-20    100           2            0
2014-03-21     88           1            0
2014-03-22     81           1            1
2014-03-23     77           1            0
2014-03-24     91           1            0
2014-03-25     86           1            0
2014-03-26     89           1            0
2014-03-27     94           1            0
2014-03-28     88           1            0
2014-03-29     78           1            0
2014-03-30     74           1            0
2014-03-31     92           1            0
2014-04-01     84           1            0
2014-04-02     83           1            0
2014-04-03     85           1            0
2014-04-04     85           1            0
2014-04-05     74           1            1
2014-04-06     72           1            0
2014-04-07     86           1   

In [41]:
print(cats[18001])
batches

['globe', 'matchstick', 'watercooler', 'art', 'photography']


[array(['globe', 'matchstick', 'watercooler'], dtype='<U11'),
 array(['art', 'photography'], dtype='<U11')]

In [13]:
kws = ['asteroid', 'world', 'earth']

g_trends = TrendReq(tz=360, timeout=(5,25), proxies=proxy_list, retries=2, backoff_factor=0.1)
g_trends.build_payload(kws, timeframe = '2013-07-22'+' '+'2013-08-20', geo='', gprop='')
trend = g_trends.interest_over_time()
trend = trend.drop(columns=['isPartial'])

,asteroid,world,earth,isPartial
date,,,,
2013-07-22,0,84,17,False
2013-07-23,0,84,17,False
2013-07-24,0,83,17,False
2013-07-25,0,83,17,False
2013-07-26,0,84,17,False
2013-07-27,0,90,18,False
2013-07-28,0,96,19,False
2013-07-29,0,85,17,False
2013-07-30,0,83,16,False


In [17]:
#trend = trend.drop(columns=['isPartial'])
trend

,asteroid,world,earth
date,,,
2013-07-22,0,84,17
2013-07-23,0,84,17
2013-07-24,0,83,17
2013-07-25,0,83,17
2013-07-26,0,84,17
2013-07-27,0,90,18
2013-07-28,0,96,19
2013-07-29,0,85,17
2013-07-30,0,83,16


In [11]:
batch_num = int(len(cats[0])/5)
if len(cats[0])%5 > 0:
    batch_num = batch_num + 1

np.array(np.array_split(cats[0], batch_num)).tolist()

[['asteroid', 'world', 'earth'], ['u.s.', 'space', 'challenge']]

In [ ]:
date = get_date(dataset["Page content"][start])
tmp = [0]
batch_num = len(cats[0])/5
if len(cats[0])%5 > 0:
    batch_num = batch_num + 1
search_list = []
group = []
for i in range(len(cats[0])):
    if i%5==0:
        if i not 0:
            search.append(group)
        group = []
    else:
        group.append(cats[0][i])

for cat in cats[0]:
    print(colored(str(i) + " Search: " + cat, "red"))
    tmp.append(trend_rate(cat, date))
print(max(tmp))

In [ ]:
date = get_date(dataset["Page content"][start])
print(date)

In [ ]:
date = get_date(dataset["Page content"][start])
tmp = [0]
for cat in cats[0]:
    print(colored(str(i) + " Search: " + cat, "red"))
    tmp.append(trend_rate(cat, date))
print(max(tmp))

In [ ]:


#start = 18051

    
res = []
for i in range(start, end):
    try:
        try:
            date = get_date(dataset["Page content"][i])
            tmp = [0]
            for cat in cats[i]:
                print(colored(str(i) + " Search: " + cat, "red"))
                tmp.append(trend_rate(cat, date))
            res.append(max(tmp))
        except:
            res.append(1)
        print(colored(str(i)+" done", "green"))
        if i%5 == 0:
            export(i+1)
            #time.sleep(30)
        if i == end-1:
            export(end)
    except Exception as e:
        print(e)
        export(i)
        print("Break at: " + str(i))

In [ ]:
#res[1] = 2 0 o
n = 0
print(np.mean(res))
for i in range(start, start+len(res)):
    if (res[i] > float(np.mean(res))) and (1 == dataset["Popularity"][i]):
        n = n+1
    elif (res[i] < float(np.mean(res))) and (-1 == dataset["Popularity"][i]):
        n = n+1
    print(str(i) + ": \t" + str(dataset["Popularity"][i]) +"\t"+str(res[i]))
print("accu: " + str(n/len(res)))

In [ ]:
# def export():
#     g_trend = pd.DataFrame(res)
#     g_trend.columns = ["trend_rate"]
#     ID = [i for i in range(start, end)]
#     g_trend['Id'] = ID
#     g_trend = g_trend.set_index('Id')
#     g_trend.to_csv("trend.csv")

In [ ]:
"trend_%s_%s.csv" % (str(start), str(55))